In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [21]:
#data preparation and feature engineering
df = pd.read_csv('customer_churn.csv')
df.columns = df.columns.str.lower().str.replace(' ','_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ','_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [22]:
senior = {
    0: 'no',
    1: 'yes'
}
df.seniorcitizen = df.seniorcitizen.map(senior)

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,no,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,no,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0


In [24]:
#split the dataset using the 60%-20%-20% rule
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [35]:
#identify the numerical and categorical features
numerical = ['tenure', 'monthlycharges','totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner', 
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection', 
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
    
]

In [39]:
#write the training function
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    return dv, model

In [40]:
#prediction function
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    
    return y_pred

In [48]:
C=1.0

In [49]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)
y_test = df_test.churn.values

auc = roc_auc_score(y_test, y_pred)
auc

0.8583300833089582

###### Save the model

In [50]:
import pickle

In [51]:
output_file = f'model_C={C}.bin'

In [52]:
output_file

'model_C=1.0.bin'

In [53]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

###### Load Model

In [58]:
test_customer = df_test.iloc[4].to_dict()
test_customer

{'customerid': '7919-zodzz',
 'gender': 'female',
 'seniorcitizen': 'no',
 'partner': 'yes',
 'dependents': 'yes',
 'tenure': 10,
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'no',
 'streamingtv': 'no',
 'streamingmovies': 'yes',
 'contract': 'one_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'mailed_check',
 'monthlycharges': 65.9,
 'totalcharges': 660.05,
 'churn': 0}